## Extract Job Posts from Indeed

### Create a table in database

In [22]:
import pandas
import configparser
import psycopg2

Read the database connection info from the config.ini

In [23]:
config = configparser.ConfigParser()
config.read('config.ini')

host = config['myaws']['host']
db = config['myaws']['database']
user = config['myaws']['user']
pwd = config['myaws']['pwd']

Establish a connection to the database, and create a cursor.

In [24]:
conn = psycopg2.connect(host = host,
                       user = user,
                        password = pwd,
                        dbname = db
                       )
cur = conn.cursor()

Design the table in SQL

In [25]:
table_sql = """
            CREATE TABLE IF NOT EXISTS indeed_gp8
            (
                id SERIAL,
                job_title VARCHAR(200),
                job_company VARCHAR(200),
                job_loc VARCHAR(200),
                job_salary VARCHAR(200),
                job_summary TEXT,
                PRIMARY KEY(id)
            );

            """

Create the table

In [26]:
cur.execute(table_sql)
conn.commit()

### Request HTML

urllib.request makes simple HTTP requests to visit a web page and get the content via the Python standard library. Here we define the URL to search job posts about intelligence analysts.

In [27]:
url = 'https://www.indeed.com/jobs?q=intelligence+analyst&start=0'

In [28]:
import urllib.request
response = urllib.request.urlopen(url)
html_data= response.read()
#print(html_data.decode('utf-8'))

## Parese HTML

We can use the inspector tool in browsers to analyze webpages and use beautifulsoup to extract webpage data. pip install the beautiful soup if needed.

In [29]:
!pip install beautifulsoup4

In [30]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html_data,'html.parser')
#print (soup)

Use the tag.find_all(‘tag_name’, tage_attr = ‘possible_value’) function to return a list of tags where the attribute equals the possible_value.

Common attributes include:
id
class_

Common functions include:
tag.text: return the visible part of the tag
tag.get(‘attribute’): return the value of the attribute of the tag

Since all the job posts are in the div tag class = 'jobsearch-Sprep...', we need to find that div tag from the body tag.

In [31]:
for table_resultsBody in soup.find_all('table', id = 'resultsBody'):
    pass
    #print(table_resultsBody)

In [32]:
for table_pageContent in table_resultsBody.find_all('table', id = 'pageContent'):
    pass
    #print(table_pageContent)

In [33]:
for td_resultsCol in table_pageContent.find_all('td', id = 'resultsCol'):
    pass
    #print(td_resultsCol)

## Save Data to Database

Now we find the div tag contains the job posts. We need to identify the job title, company, ratings, reviews, salary, and summary. We can save those records to our table in the database.

In [34]:
#  identify the job title, company, ratings, reviews, salary, and summary
for div_row in td_resultsCol.find_all('div', class_='jobsearch-SerpJobCard unifiedRow row result'):

    # find job title
    job_title = None
    job_company  = None
    job_rating = None
    job_loc = None
    job_salary = None
    job_summary = None
    for div_title in div_row.find_all('div', class_ = 'title'):
        job_title = div_title.text.strip().replace("'","_")
    
    for div_dsc in div_row.find_all('div', class_ = 'sjcl'):
        
        #find company name
        for span_company in div_dsc.find_all('span', class_ = 'company'):
            job_company = span_company.text.strip().replace("'","_")
        
        # find location
        for div_loc in div_dsc.find_all('div', class_ = 'location accessible-contrast-color-location'):
            job_loc = div_loc.text.strip().replace("'","_")
    
    # find salary
    for div_salary in div_row.find_all('div',class_ ='salarySnippet'):
        job_salary = div_salary.text.strip().replace("'","_")
    
    #find summary
    for div_summary in div_row.find_all('div', class_ = 'summary'):
        job_summary = div_summary.text.strip().replace("'","_")
        
    # insert into database
    
    sql_insert = """
                insert into indeed_gp8(job_title,job_company,job_loc,job_salary,job_summary)
                values('{}','{}','{}','{}','{}')            
                """.format(job_title,job_company,job_loc,job_salary,job_summary)

    cur.execute(sql_insert)
    conn.commit()

## View the Table

In [35]:
df = pandas.read_sql_query('select * from indeed_gp8', conn)
df[:]

,id,job_title,job_company,job_loc,job_salary,job_summary
0,1,Intelligence Analyst,National Security Agency,"Fort Meade, MD","$47,016 - $74,759 a year",The National Security Agency (NSA) is part of ...
1,2,Quantitative Analyst / Developer,RT Signal,"New York, NY 10013 (Tribeca area)","$95,000 - $165,000 a year","We are a small, but well-funded (financially s..."
2,3,H-ISAC Cyber Threat Intelligence Analyst,Perch Security,"Titusville, FL","$50,000 - $75,000 a year",Ability to communicate technical issues to tec...
3,4,"Cybersecurity Analyst (DLP) (60-70k, US Citize...",Maveris,"Martinsburg, WV","$60,000 - $70,000 a year",The Analyst can identify risks in security sys...
4,5,Transportation Analyst,Lipman Family Farms,"Immokalee, FL 34142",None,"Perform fact based, data driven analysis, and ..."
5,6,Security Analyst,Seen by Indeed,"Washington, DC",None,Get matched – Companies apply to you with oppo...
6,7,Intelligence Analyst,Booz Allen Hamilton,None,None,We’re an EOE that empowers our people—no matte...
7,8,Intelligence Analyst I - MRS,WorldAware,None,None,WorldAware is dedicated to equal opportunity i...
8,9,Intelligence Analyst,BAE Systems,None,None,Having an understanding of physical and techni...
9,10,Intelligence Analyst,5 Stones intelligence,None,None,Analyze threat information from multiple sourc...


In [36]:
cur.close()
conn.close()